In [1]:
% matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import pickle

from datetime import datetime, date, time
from textblob import TextBlob
from textblob import Word
from pre_processing import *
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to C:\Users\Andres
[nltk_data]     Montero\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
stop = stopwords.words('english')
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]


In [3]:
DATA_FOLDER = './data'

tweets_col_names=['text']

tweets_dtypes = {'text': str }

cleaned_pos = True
cleaned_neg = True
cleaned_test = False

In [4]:
tweets_neg = pd.read_fwf(DATA_FOLDER + '/train_neg.txt',  names=tweets_col_names,
                              dtype=tweets_dtypes)
tweets_neg.head(20)

,text
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...
1,glad i dot have taks tomorrow ! ! #thankful #s...
2,1-3 vs celtics in the regular season = were fu...
3,<user> i could actually kill that girl i'm so ...
4,<user> <user> <user> i find that very hard to ...
5,wish i could be out all night tonight ! <user>
6,<user> i got kicked out the wgm
7,rt <user> <user> <user> yes she is ! u tell it...
8,why is she so perfect <url>
9,<user> hi harry ! did u havea good time in aus...


In [5]:
tweets_neg.index = tweets_neg.index +1
tweets_neg.drop_duplicates(inplace=True)


In [6]:
tweets_neg

,text
1,vinco tresorpack 6 ( difficulty 10 of 10 objec...
2,glad i dot have taks tomorrow ! ! #thankful #s...
3,1-3 vs celtics in the regular season = were fu...
4,<user> i could actually kill that girl i'm so ...
5,<user> <user> <user> i find that very hard to ...
6,wish i could be out all night tonight ! <user>
7,<user> i got kicked out the wgm
8,rt <user> <user> <user> yes she is ! u tell it...
9,why is she so perfect <url>
10,<user> hi harry ! did u havea good time in aus...


In [7]:
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: filter_digits(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: separate_hash(x))
tweets_neg['text'] = tweets_neg['text'].str.replace('#', '')
tweets_neg['text'] = tweets_neg['text'].str.replace('ur', 'your')
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: expand_contractions(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: interpret_emoji(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: remove_punctuation(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: remove_words(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: replace_moreletters(x))
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: split_number_text(x))

tweets_neg.head()

,text
1,vinco tresorpack difficulty of object disassem...
2,glad i dot have taks tomorrow thankful startho
3,vs celtics in the regular season were fucked i...
4,i could actually kill that girl i am so sorry
5,i find that very hard to believe im afraid


In [8]:
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
tweets_neg['text'].head()

1    vinco tresorpack difficulty object disassemble...
2              glad dot taks tomorrow thankful startho
3       vs celtics regular season fucked play playoffs
4                       could actually kill girl sorry
5                          find hard believe im afraid
Name: text, dtype: object

### Remove more comon words of the data set

In [9]:
freq_max = pd.Series(' '.join(tweets_neg['text']).split()).value_counts()[:10]
freq_max

yourl     28483
frame      6690
rt         4225
like       4093
get        3756
yoyour     3698
go         3662
one        3336
want       3259
miss       3110
dtype: int64

In [10]:
freq_max = list(freq_max.index)
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max))
tweets_neg['text'].head()

1    vinco tresorpack difficulty object disassemble...
2              glad dot taks tomorrow thankful startho
3       vs celtics regular season fucked play playoffs
4                       could actually kill girl sorry
5                          find hard believe im afraid
Name: text, dtype: object

In [11]:
freq_min = pd.Series(' '.join(tweets_neg['text']).split()).value_counts()[-10:]
freq_min

saidd            1
worstfeelings    1
agrochemicals    1
panacea          1
bravado          1
automat          1
ricki            1
ntrl             1
ahwell           1
somewheres       1
dtype: int64

In [12]:
freq_min = list(freq_min.index)
tweets_neg['text'] = tweets_neg['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min))
tweets_neg['text'].head()

1    vinco tresorpack difficulty object disassemble...
2              glad dot taks tomorrow thankful startho
3       vs celtics regular season fucked play playoffs
4                       could actually kill girl sorry
5                          find hard believe im afraid
Name: text, dtype: object

### Correct spelling, takes a long time ! for now we will coment it and use it later

In [13]:
# tweets_neg['text'] = tweets_neg['text'].apply(lambda x: ''.join(TextBlob(x).correct()))
# tweets_neg

### Tokenization

In [14]:
# tweets_neg['text'] = tweets_neg['text'].apply(lambda x: (TextBlob(x).words))

In [15]:
# tweets_neg.head()

### Lematization

In [16]:
tweets_neg['text_lema'] = tweets_neg['text'].apply( lemmatize_text)
tweets_neg

,text,text_lema
1,vinco tresorpack difficulty object disassemble...,"[vinco, tresorpack, difficulty, object, disass..."
2,glad dot taks tomorrow thankful startho,"[glad, dot, taks, tomorrow, thankful, startho]"
3,vs celtics regular season fucked play playoffs,"[v, celtic, regular, season, fucked, play, pla..."
4,could actually kill girl sorry,"[could, actually, kill, girl, sorry]"
5,find hard believe im afraid,"[find, hard, believe, im, afraid]"
6,wish could night tonight,"[wish, could, night, tonight]"
7,got kicked wgm,"[got, kicked, wgm]"
8,yes tell lips closed okay,"[yes, tell, lip, closed, okay]"
9,perfect,[perfect]
10,hi harry havea good time aus didnt see maybe n...,"[hi, harry, havea, good, time, au, didnt, see,..."


## Save pickle

In [17]:
if not cleaned_neg :
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_neg.pickle","wb")
    pickle.dump(tweets_neg['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
else:
    print("Already Trained!")

Already Trained!


In [18]:
pickle_in = open("data/pre_processed/tweets_neg.pickle","rb")
test_pickle = pickle.load(pickle_in)
test_pickle

0        [into, tresorpack, difficulty, object, disasse...
1           [glad, dot, take, tomorrow, thankful, started]
2        [v, celtic, regular, season, tucked, play, pla...
3                     [could, actually, kill, girl, sorry]
4                        [find, hard, believe, in, afraid]
5                            [wish, could, night, tonight]
6                                        [got, kicked, wm]
7                           [yes, tell, lip, closed, okay]
8                                                [perfect]
9        [hi, harry, have, good, time, a, didn, see, ma...
10       [introduction, programming, +, +, nd, edition,...
13                                              [white, a]
14                      [dan, love, sad, wheresthegeneral]
15                  [many, wonderful, building, do, still]
16                        [annoying, secretly, find, good]
17       [post, boom, spanish, american, fiction, sun, ...
18       [layer, heart, paperback, journey, inspired, r.

In [19]:
tweets_pos = pd.read_fwf(DATA_FOLDER + '/train_pos.txt',  names=tweets_col_names,
                              dtype=tweets_dtypes)
tweets_pos.index = tweets_pos.index + 1
tweets_pos.drop_duplicates(inplace=True)
tweets_pos

,text
1,<user> i dunno justin read my mention or not ....
2,"because your logic is so dumb , i won't even c..."
3,""" <user> just put casper in a box ! "" looved t..."
4,<user> <user> thanks sir > > don't trip lil ma...
5,visiting my brother tmr is the bestest birthda...
6,<user> yay ! ! #lifecompleted . tweet / facebo...
7,<user> #1dnextalbumtitle : feel for you / roll...
8,workin hard or hardly workin rt <user> at hard...
9,<user> i saw . i'll be replying in a bit .
10,this is were i belong


In [20]:
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: filter_digits(x))
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: separate_hash(x))
tweets_pos['text'] = tweets_pos['text'].str.replace('#', '')
tweets_pos['text'] = tweets_pos['text'].str.replace('ur', 'your')
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: expand_contractions(x))
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: interpret_emoji(x))
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: remove_punctuation(x))
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: remove_words(x))
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: replace_moreletters(x))
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: split_number_text(x))

In [21]:
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [22]:
freq_max_pos = pd.Series(' '.join(tweets_pos['text']).split()).value_counts()[:10]
freq_max_pos = list(freq_max_pos.index)
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_pos))

In [23]:
freq_min_pos = pd.Series(' '.join(tweets_pos['text']).split()).value_counts()[-10:]
freq_min_pos = list(freq_min_pos.index)
tweets_pos['text'] = tweets_pos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_pos))

In [24]:
# tweets_pos['text'] = tweets_pos['text'].apply(lambda x: ''.join(TextBlob(x).correct()))

In [25]:
tweets_pos['text_lema'] = tweets_pos['text'].apply( lemmatize_text)
tweets_pos

,text,text_lema
1,dunno justin read mention justin god knows hop...,"[dunno, justin, read, mention, justin, god, kn..."
2,logic dumb even crop name photo tsk,"[logic, dumb, even, crop, name, photo, tsk]"
3,put casper box looved battle crakkbitch,"[put, casper, box, looved, battle, crakkbitch]"
4,thanks sir trip lil mama keep doin ya thang,"[thanks, sir, trip, lil, mama, keep, doin, ya,..."
5,visiting brother tmr bestest birthday gift eveerr,"[visiting, brother, tmr, bestest, birthday, gi..."
6,yay lifecompleted tweet facebook let please,"[yay, lifecompleted, tweet, facebook, let, ple..."
7,dnextalbumtitle feel rollercoaster life song c...,"[dnextalbumtitle, feel, rollercoaster, life, s..."
8,workin hard hardly workin hardee's futyoure co...,"[workin, hard, hardly, workin, hardee's, futyo..."
9,saw replying bit,"[saw, replying, bit]"
10,belong,[belong]


In [26]:
if not cleaned_pos:  
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/tweets_pos.pickle","wb")
    pickle.dump(tweets_pos['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
else:
    print("Already Trained!")

Already Trained!


In [27]:
pickle_in = open("data/pre_processed/tweets_pos.pickle","rb")
test_pickle_pos = pickle.load(pickle_in)
test_pickle_pos

0        [funny, austin, read, mention, austin, god, kn...
1              [logic, dumb, even, crop, name, photo, ask]
2            [put, caper, box, looked, battle, crakkbitch]
3        [thanks, sir, trip, ll, mamma, keep, down, a, ...
4        [visiting, brother, mr, beset, birthday, gift,...
5        [may, lifecompleted, sweet, facebook, let, ple...
6        [dnextalbumtitle, feel, rollercoaster, life, s...
7        [working, hard, hardly, working, harder's, fut...
8                                     [saw, replying, bit]
9                                                 [belong]
10                                  [and, cheer, national]
11       [send, invitation, shop, line, find, everythin...
12                                [woke, finn, go, church]
13                                [agreed, day, left, the]
14                                       [money, katemelo]
15                   [dam, alexis, got, lot, say, twitter]
16       [grateful, today, dream, fulfilled, heart, ful.

In [28]:
# test_data = pd.read_csv(DATA_FOLDER + '/test_data.txt',  names=tweets_col_names,
#                               dtype=tweets_dtypes)
# test_data.drop_duplicates(inplace=True)
# test_data.head()
# test_data = pd.read_fwf(DATA_FOLDER + '/test_data.txt',  names=tweets_col_names,
#                               dtype=tweets_dtypes)
test_data = pd.read_fwf(DATA_FOLDER + '/test_data.txt', header=None)
test_data.index = test_data.index +1
test_data.drop_duplicates(inplace=True)
test_data.drop(columns=[1,2], inplace=True)
test_data.rename(columns={0:'text'}, inplace= True)
test_data

,text
1,"1,sea doo pro sea scooter ( sports with the po..."
2,"2,<user> shucks well i work all week so now i ..."
3,"3,i cant stay away from bug thats my baby"
4,"4,<user> no ma'am ! ! ! lol im perfectly fine ..."
5,"5,whenever i fall asleep watching the tv , i a..."
6,"6,<user> he needs to get rid of that thing ! i..."
7,"7,its whatever . in a terrible mood ( ("
8,"8,yesss ! rt <user> <user> thanks jordan , i l..."
9,"9,my friend <user> text me to check up on me l..."
10,"10,<user> #followback please . when will ur #u..."


In [29]:
test_data['text'] = test_data['text'].apply(lambda x: filter_digits(x))
test_data['text'] = test_data['text'].apply(lambda x: separate_hash(x))
test_data['text'] = test_data['text'].str.replace('#', '')
test_data['text'] = test_data['text'].str.replace('ur', 'your')
test_data['text'] = test_data['text'].apply(lambda x: expand_contractions(x))
test_data['text'] = test_data['text'].apply(lambda x: interpret_emoji(x))
test_data['text'] = test_data['text'].apply(lambda x: remove_punctuation(x))
test_data['text'] = test_data['text'].apply(lambda x: remove_words(x))
test_data['text'] = test_data['text'].apply(lambda x: replace_moreletters(x))
test_data['text'] = test_data['text'].apply(lambda x: split_number_text(x))

In [30]:
test_data['text'] = test_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [31]:
freq_max_test = pd.Series(' '.join(test_data['text']).split()).value_counts()[:10]
freq_max_test = list(freq_max_test.index)
test_data['text'] = test_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_max_test))

In [32]:
freq_min_test = pd.Series(' '.join(test_data['text']).split()).value_counts()[-10:]
freq_min_test = list(freq_min_test.index)
test_data['text'] = test_data['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_min_test))

In [33]:
test_data['text'] = test_data['text'].apply(lambda x: ''.join(TextBlob(x).correct()))

In [34]:
test_data['text_lema'] = test_data['text'].apply( lemmatize_text)
test_data

,text,text_lema
1,sea do pro sea sooner sports portable sea do s...,"[sea, do, pro, sea, sooner, sport, portable, s..."
2,sucks well work week come cheer oh put batteri...,"[suck, well, work, week, come, cheer, oh, put,..."
3,can stay away bug that baby,"[can, stay, away, bug, that, baby]"
4,madam in perfectly fine contagious anymore may,"[madam, in, perfectly, fine, contagious, anymo..."
5,whenever fall asleep watching to always wake h...,"[whenever, fall, asleep, watching, to, always,..."
6,needs get rid thing scars need car either need...,"[need, get, rid, thing, scar, need, car, eithe..."
7,whatever terrible mood,"[whatever, terrible, mood]"
8,yes thanks jordan donna call later,"[yes, thanks, jordan, donna, call, later]"
9,friend text check last night,"[friend, text, check, last, night]"
10,followback please unitytoyour come europe sweden,"[followback, please, unitytoyour, come, europe..."


In [35]:
if not cleaned_test:  
    print("Saving file with preprocessed Tweets")
    pickle_out = open("data/pre_processed/test_data.pickle","wb")
    pickle.dump(test_data['text_lema'], pickle_out)
    pickle_out.close()
    print("Saved!")
else:
    print("Already Trained!")

Saving file with preprocessed Tweets
Saved!


In [36]:
pickle_in = open("data/pre_processed/test_data.pickle","rb")
test_data_pickle = pickle.load(pickle_in)
test_data_pickle

1        [sea, do, pro, sea, sooner, sport, portable, s...
2        [suck, well, work, week, come, cheer, oh, put,...
3                       [can, stay, away, bug, that, baby]
4        [madam, in, perfectly, fine, contagious, anymo...
5        [whenever, fall, asleep, watching, to, always,...
6        [need, get, rid, thing, scar, need, car, eithe...
7                               [whatever, terrible, mood]
8                [yes, thanks, jordan, donna, call, later]
9                       [friend, text, check, last, night]
10       [followback, please, unitytoyour, come, europe...
11            [watch, dumb, mass, get, lock, today, happy]
12       [obsessed, phase, killed, best, album, ever, n...
13       [robert, de, no, gay, name, vewy, would, under...
14                 [canada, grade, since, grade, th, suck]
15        [please, say, hi, denmark, would, amazing, live]
16                                         [finally, home]
17       [custom, picture, poster, wide, complete, gold.